# RAG와 Memory Chain을 모두 적용한 챗봇 만들기를 위한 연습입니다🙂
# Chabots
 - llm에 대한 가장 인기있는 사용케이스
 - 오랜기간의 학습을 거치며, 관련 정보를 이용, 현재상황에 대한 대화를 나눌 수 있음

# Atchitecture
 - Chatbot을 디자인하기 위해서는 여러 요소를 고려해야함
 - 당연히 DB, Memory 등 모든 정보를 넣고 prompt를 제공하면 성능은 높아지지만 속도, 자원 등 trade-off를 생각해야함
 - 아래는 일반적인 chatbot의 아키텍처
 ![Chabot Architecture](./imgs/Chatbot_Architecture.png)



# Chatbot의 기본 구성요소
 - Chatmodels : 사용자 쿼리 및 제공 데이터를 가지고 텍스트를 생성할 LLM모델
 - PromptTemplates : 모델이 사용자 요구사항을 잘 받아들이게 하고, 출력물의 결과를 사용자가 원하는 형태로 만들기 위한 template
 - ChatHistory : 사용자와의 대화 기록을 저장, 다음 답변 시 이전의 대화를 바탕으로 답변을 생성할 수 있음
 - Retrievers : 사용자의 쿼리를 통해 적정한 데이터를 찾아서 Chatmodels에게 제공해주는 역할
 

In [196]:
from dotenv import load_dotenv
load_dotenv('../dot.env')

True

In [197]:
# 우선은 GPT 모델을 사용해봅니다
from langchain_openai import ChatOpenAI

# chat = ChatOpenAI(
#                    base_url="http://localhost:1234/v1",
#                    api_key="lm-studio",
#                    model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
#                    temperature = 0.0,
#                    )
chat = ChatOpenAI(
                   base_url="http://localhost:1234/v1",
                   api_key="lm-studio",
                   model="cognitivecomputations/dolphin-2.9-llama3-8b-gguf",
                   temperature = 0.0,
                   )


In [198]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage as AIM

chat.invoke(
    [HumanMessage(content = "Translate this sentence from English to French: I love programming.")])

#질의 형식 보기


AIMessage(content='Je adore programmer.', response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 4, 'total_tokens': 8}, 'model_name': 'cognitivecomputations/dolphin-2.9-llama3-8b-gguf', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-aac46aed-ea59-48e7-b302-ebe2e582d853-0')

# 위 상태의 챗봇은 이전 대화를 기억하지 못합니다.
 - 가장 간단한 방식으로 이전의 대화를 기억하게끔 chatbot을 구성해보겠습니다.

Human Message와 AIMessage가 섞여있을 때 Chatbot은 가장 최근의 Human Message에 대해 대화를 진행합니다.

In [199]:
AIMessage = chat.invoke(
    [
        HumanMessage(content = "Translate this sentence from English to French: I love programming."),
        AIM(content = "J'aime le codage"),
        HumanMessage(content = "What did you just say?")
    ]
)

AIMessage.content

'I said "J\'aime le codage" which means "I love programming" in French.\n'

# PromptTemplate을 주면 Chatbot이 더욱 효율적으로 과제를 알아들을 수 있습니다.
- 우리는 model에 template을 넣는 방식으로 이를 해결할 수 있습니다

In [200]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# prompt는 모델에 잘 입력될 수 있는 형태로 정의해야합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            "You are a helpful assistant. Answer all questions to the best of your ability."
        ),
        # MessagesPlaceholder는 대화의 이전 메시지들을 포함하는 변수입니다. 이를 통해 챗봇은 이전 대화의 맥락을 이해하고, 연속적인 대화를 진행할 수 있습니다. 'messages'라는 변수 이름으로 이전 메시지들을 참조하게 됩니다.
        MessagesPlaceholder(variable_name = "messages")
])
chain = prompt | chat

# Message들 자체를 메타데이터로 주어 기억하게 할 수도 있습니다.
- MessagePlaceholder는 대화의 이전 메시지들을 포함하는 변수입니다. 이를 통해 챗봇은 이전 대화의 맥락을 이해하고, 연속적인 대화를 진행할 수 있습니다. 'messages'라는 변수 이름으로 이전 메시지들을 참조하게 됩니다.



In [201]:
chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate this sentence from English to French: I love programming."),
            AIM(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
).content

'I said, "J\'adore la programmation," which translates to "I love programming" in English.\n'

# Message history
- 채팅 기록을 관리하기 위한 간편한 방법으로, 채팅 메시지를 저장하고 불러오는 역할을 하는 MessageHistory 클래스를 사용할 수 있습니다. 
- 다양한 데이터베이스에 메시지를 지속적으로 저장하는 많은 내장 메시지 기록 통합 기능이 있지만, 이 빠른 시작 가이드에서는 메모리 내에서 작동하는 데모 메시지 기록인 ChatMessageHistory를 사용하겠습니다.


In [202]:
#ChatMessagehistory를 이용하면 간단한 방식으로 Memory를 관리할 수 있습니다 가령,
from langchain.memory import ChatMessageHistory
#인스턴스 생성
chathistory = ChatMessageHistory()
#add_user_message()메서드를 이용한 유저메시지 삽입
chathistory.add_user_message('hi')
#add_ai_message()메서드를 이용한 유저메시지 삽입
chathistory.add_user_message("what's up?")

chathistory.messages

[HumanMessage(content='hi'), HumanMessage(content="what's up?")]

# 생성한 chatmessagehistory를 chain의 Messageplaceholder에 넣어 대화를 진행할 수 있습니다.

In [203]:
chathistory.add_user_message(
    "Summarize the conversation in bullet points."
)

response = chain.invoke({"messages": chathistory.messages})

response.content

"- Hi\n- What's up?\n"

# 위에서 만든 챗봇을 이용해 RAG를 적용해보겠습니다.

In [204]:
# 여기서는 webbaseloader를 이용해 인터넷에 있는 문서를 가져오도록 하겠습니다.
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
data = loader.load()

In [205]:
# 가져온 데이터는 대부분의 경우 매우 많은 양의 데이터를 가지고 있습니다.
data

[Document(page_content='\n\n\n\n\nGet started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it\'s not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace‚ÄãWe provide multiple ways to log 

# Splitter
많은 양의 문서를 한번에 context로 제공하는 것은 많은 자원을 요구하거나 불가능한 경우가 많습니다.

In [206]:
# 재귀 문자 분할기를 이용해 문서를 나누어서 제공할 수 있습니다.
# 문서하나의 사이즈는 500이며, 겹치는 부분은 없게 설정했습니다.
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

# Chroma DB를 이용, vector store를 만듭니다

In [207]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

# Retriever 생성
- 관련있는 문서는 4개를 가져오도록 합니다.

In [208]:
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("how can langsmith help with testing?")

docs

[Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'}),
 Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏

# Local LLM을 통해 만든 Chatbot에 prompt를 엮어줍니다.

In [209]:
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context} and must answer in Korean",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

# chathistory와 Retriever를 context로 갖는 chain을 구성했습니다.

In [210]:
document_chain.__dict__

{'name': None,
 'bound': RunnableBinding(bound=RunnableAssign(mapper={
   context: RunnableLambda(format_docs)
 }), config={'run_name': 'format_inputs'})
 | ChatPromptTemplate(input_variables=['context', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Answer the user's questions based on the below context:\n\n{context} and must answer in Korean")), MessagesPlaceholder(variable_name='messages')])
 | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x32645a8d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x3257d1710>, model_name='cognitivecomputations/dolphin

In [211]:
from langchain.memory import ChatMessageHistory

chathistory = ChatMessageHistory()

chathistory.add_user_message("how can langsmith help with testing?")

document_chain.invoke(
    {
        "messages": chathistory.messages,
        "context": docs,
    }
)

'LangSmith는 테스트를 위한 도구로, 개발자들이 코드를 작성하고 이를 자동으로 실행하여 결과를 확인할 수 있는 기능을 제공합니다. 또한, 입력 데이터와 출력 데이터를 비교하는 기능을 포함하여, 코드가 기대를 만족시키는지 확인할 수 있는 데모를 쉽게 생성할 수 있습니다.'

# 검색 체인 생성
- 다음으로, 검색기를 체인에 통합합시다. 
- 검색기는 사용자가 전달한 마지막 메시지와 관련된 정보를 검색해야 하므로, 이를 추출하여 입력으로 사용하고 관련 문서를 가져와 현재 체인의 컨텍스트로 추가합니다. 이 컨텍스트와 이전 메시지들을 문서 체인에 전달하여 최종 답변을 생성합니다.

- 또한, RunnablePassthrough.assign() 메소드를 사용하여 각 호출 시 중간 단계를 전달합니다. 다음과 같이 보입니다:


In [212]:
from typing import Dict

from langchain_core.runnables import RunnablePassthrough

# 마지막 사용자 메시지 내용을 추출하는 함수
def parse_retriever_input(params: Dict):
    return params["messages"][-1].content

# 검색 체인을 구성하고, 문서 체인에 연결
retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever,  # 마지막 메시지를 검색기에 전달
).assign(
    answer=document_chain,  # 검색 결과를 문서 체인에 전달하여 답변 생성
)

In [213]:
# Ai의 답변을 추가하기 위해 먼저 res
response = retrieval_chain.invoke(
    {'messages' : chathistory.messages}
    )

chathistory.add_ai_message(response['answer'])
chathistory.add_user_message("좀더 자세히 말해봐")

response = retrieval_chain.invoke(
    {'messages' : chathistory.messages}
    )
response

{'messages': [HumanMessage(content='how can langsmith help with testing?'),
  AIMessage(content='LangSmith는 테스트를 위한 도구로, 개발자들이 코드를 작성하고 이를 자동으로 실행하여 결과를 확인할 수 있는 기능을 제공합니다. 또한, 입력 데이터와 출력 데이터를 비교하는 기능을 포함하여, 코드가 기대를 만족시키는지 확인할 수 있는 데모를 쉽게 생성할 수 있습니다.'),
  HumanMessage(content='좀더 자세히 말해봐')],
 'context': [Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'}),
  Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applic

In [215]:
print("context : ",response['context'],'\n')
print("answer : ",response['answer'])

context :  [Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'}), Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \

# 위 과정을 통해 chatbot에 메모리와 context를 추가하는 방법을 알아보았습니다.
- 아래 코드처럼 assign을 한 번 더 호출하지 않고 |를 이용하여 document_chain을 넣을 수도 있습니다

In [216]:
retrieval_chain_with_only_answer = (
    RunnablePassthrough.assign(
        context=parse_retriever_input |
                retriever,
    )
    | document_chain
)

retrieval_chain_with_only_answer.invoke(
    {
        "messages": chathistory.messages,
    },
)

'LangSmith는 개발자들이 코드를 작성하고 이를 자동으로 실행하여 결과를 확인할 수 있는 기능을 제공합니다. 또한, 입력 데이터와 출력 데이터를 비교하는 기능을 포함하여, 코드가 기대를 만족시키는지 확인할 수 있는 데모를 쉽게 생성할 수 있습니다. 이 기능은 개발자들이 코드를 테스트하고 버그를 찾는데 도움이 됩니다.\n'

# Query transformation
- 여기서 다룰 마지막 최적화에 대해 설명하겠습니다 
- 위 예제에서, '좀 더 자세히 말해줘!'라는 후속 질문을 했을 때, 검색된 문서에 테스트에 대한 정보가 직접적으로 포함되어 있지 않다는 것을 알 수 있습니다.
- 이는 '좀 더 자세히 말해줘!'라는 질문을 검색기에 그대로 쿼리로 전달하기 때문입니다. 검색 체인의 출력은 여전히 괜찮습니다.
- 왜냐하면 문서 체인 검색 체인이 채팅 기록을 바탕으로 답변을 생성할 수 있기 때문이지만, 더 풍부하고 유익한 문서를 검색할 수도 있습니다:

In [217]:
retriever.invoke("how can langsmith help with testing?")

[Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'}),
 Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏

In [218]:
retriever.invoke("좀 더 자세히 말해줘")

[Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'}),
 Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏

# 이러한 일반적인 문제를 해결하기 위해 입력에서 참조를 제거하는 쿼리 변환 단계를 추가합시다. 다음과 같이 기존 검색기를 감싸겠습니다:

In [219]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

# We need a prompt that we can pass into an LLM to generate a transformed search query

query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
        ),
    ]
)

query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 1,
        # If only one message, then we just pass that message's content to retriever
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    # If messages, then we pass inputs to LLM chain to transform the query, then pass to retriever
    query_transform_prompt | chat | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

 - 이제 새로운 query_transforming_retriever_chain을 사용하여 이전 체인을 다시 만들어 보겠습니다. 
- 이 새로운 체인은 입력으로 dict를 받아서 검색기에 전달할 문자열을 파싱하므로, 상위 레벨에서 추가 파싱을 할 필요가 없습니다:


In [221]:
document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

demo_ephemeral_chat_history = ChatMessageHistory()

In [222]:
demo_ephemeral_chat_history.add_user_message("how can langsmith help with testing?")

response = conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages},
)

demo_ephemeral_chat_history.add_ai_message(response["answer"])

response

{'messages': [HumanMessage(content='how can langsmith help with testing?'),
  AIMessage(content='LangSmith는 테스트를 위한 도구로, 개발자들이 코드를 작성하고 이를 자동으로 실행하여 결과를 확인할 수 있는 기능을 제공합니다. 또한, 입력 데이터와 출력 데이터를 비교하는 기능을 포함하여, 코드가 기대를 만족시키는지 확인할 수 있는 데모를 쉽게 생성할 수 있습니다.')],
 'context': [Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'}),
  Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monit

In [223]:
demo_ephemeral_chat_history.add_user_message("tell me more about that!")

conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages}
)

{'messages': [HumanMessage(content='how can langsmith help with testing?'),
  AIMessage(content='LangSmith는 테스트를 위한 도구로, 개발자들이 코드를 작성하고 이를 자동으로 실행하여 결과를 확인할 수 있는 기능을 제공합니다. 또한, 입력 데이터와 출력 데이터를 비교하는 기능을 포함하여, 코드가 기대를 만족시키는지 확인할 수 있는 데모를 쉽게 생성할 수 있습니다.'),
  HumanMessage(content='tell me more about that!')],
 'context': [Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your applicati